In [3]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

In [4]:
classification = pd.read_csv("adult_data/classification.csv")
neurons = pd.read_csv("adult_data/neurons.csv")
connections = pd.read_csv("adult_data/connections.csv")

In [8]:
connections

,pre_root_id,post_root_id,neuropil,syn_count,nt_type,root_id,cell_type
0,720575940629970489,720575940631267655,AVLP_R,7,GABA,720575940629970489,NaN
1,720575940629970489,720575940620005542,AVLP_R,16,GABA,720575940629970489,NaN
2,720575940629970489,720575940626081319,AVLP_R,7,GABA,720575940629970489,NaN
3,720575940629970489,720575940652258806,AVLP_R,5,GABA,720575940629970489,NaN
4,720575940629970489,720575940624508563,AVLP_R,21,GABA,720575940629970489,NaN
...,...,...,...,...,...,...,...
3871462,720575940618564582,720575940620407572,ME_R,12,ACH,720575940618564582,TmY18
3871463,720575940645437476,720575940616370700,ME_R,5,ACH,720575940645437476,L2
3871464,720575940626870330,720575940632926765,ME_R,5,ACH,720575940626870330,Dm3
3871465,720575940623857043,720575940625360572,ME_R,10,GLUT,720575940623857043,Dm3


In [9]:
neurons

,root_id,group,nt_type,nt_type_score,da_avg,ser_avg,gaba_avg,glut_avg,ach_avg,oct_avg
0,720575940596125868,LO.LOP,ACH,0.57,0.03,0.00,0.05,0.28,0.57,0.07
1,720575940597856265,ME,ACH,0.85,0.01,0.00,0.03,0.04,0.85,0.07
2,720575940597944841,ME.LO,ACH,0.82,0.02,0.00,0.01,0.05,0.82,0.09
3,720575940598267657,ME,GABA,0.74,0.01,0.01,0.74,0.19,0.03,0.02
4,720575940599333574,ME,ACH,0.61,0.02,0.00,0.13,0.13,0.61,0.11
...,...,...,...,...,...,...,...,...,...,...
139250,720575940661335681,GNG,GABA,0.71,0.00,0.00,0.71,0.17,0.11,0.00
139251,720575940661336193,ME.LO,ACH,0.62,0.02,0.00,0.18,0.16,0.62,0.03
139252,720575940661337217,SCL.MB_ML,ACH,1.00,0.00,0.00,0.00,0.00,1.00,0.00
139253,720575940661338497,GNG,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [4]:
classification[classification["cell_type"] == "TmY15"]

,root_id,flow,super_class,class,sub_class,cell_type,hemibrain_type,hemilineage,side,nerve
757,720575940609388501,intrinsic,optic,NaN,NaN,TmY15,NaN,NaN,right,NaN
1080,720575940648552836,intrinsic,optic,NaN,NaN,TmY15,NaN,NaN,left,NaN
2035,720575940647263011,intrinsic,optic,NaN,NaN,TmY15,NaN,NaN,left,NaN
2784,720575940598267657,intrinsic,optic,NaN,NaN,TmY15,NaN,NaN,right,NaN
3973,720575940607832498,intrinsic,optic,NaN,NaN,TmY15,NaN,NaN,right,NaN
...,...,...,...,...,...,...,...,...,...,...
137893,720575940625413925,intrinsic,optic,NaN,NaN,TmY15,NaN,NaN,right,NaN
137908,720575940629775324,intrinsic,optic,NaN,NaN,TmY15,NaN,NaN,right,NaN
138030,720575940637775593,intrinsic,optic,NaN,NaN,TmY15,NaN,NaN,left,NaN
138520,720575940635907179,intrinsic,optic,NaN,NaN,TmY15,NaN,NaN,right,NaN


In [5]:
# add cell_type to connections via pre_root_id
connections = pd.merge(connections, classification[["root_id", "cell_type"]], left_on="pre_root_id", right_on="root_id")

In [6]:
# Map neuron IDs to matrix indices
unique_neurons = np.unique(connections[['pre_root_id', 'post_root_id']])
id_to_index = {id: i for i, id in enumerate(unique_neurons)}

# Convert neuron IDs in your DataFrame to matrix indices
pre_indices = connections['pre_root_id'].map(id_to_index)
post_indices = connections['post_root_id'].map(id_to_index)

# Create a sparse adjacency matrix
adjacency_matrix = coo_matrix((connections['syn_count'], (pre_indices, post_indices)),
                              shape=(len(unique_neurons), len(unique_neurons)))

# Optional: Convert to CSR format
adjacency_matrix_csr = adjacency_matrix.tocsr()

In [10]:
connections[connections["cell_type"] == "TmY15"]

,pre_root_id,post_root_id,neuropil,syn_count,nt_type,root_id,cell_type
655015,720575940626644624,720575940628219976,LOP_L,2,GABA,720575940626644624,TmY15
655016,720575940626644624,720575940653539318,LO_L,5,GABA,720575940626644624,TmY15
655017,720575940626644624,720575940653539318,ME_L,11,GABA,720575940626644624,TmY15
655018,720575940626644624,720575940646414132,LOP_L,2,GABA,720575940626644624,TmY15
655019,720575940626644624,720575940653539318,LOP_L,3,GABA,720575940626644624,TmY15
...,...,...,...,...,...,...,...
3836745,720575940628433551,720575940613961634,ME_R,4,GABA,720575940628433551,TmY15
3836746,720575940628433551,720575940613961634,LOP_R,1,GABA,720575940628433551,TmY15
3867915,720575940633700947,720575940615175842,LOP_R,3,GABA,720575940633700947,TmY15
3867916,720575940633700947,720575940615175842,LO_R,4,GABA,720575940633700947,TmY15


In [7]:
import torch
import pandas as pd
from adult_models import AdultConnectomeNetwork, IntegratedModel
from temporal_convolutional import TemporalConvNet


def get_synapse_df():
    classification = pd.read_csv("adult_data/classification.csv")
    connections = pd.read_csv("adult_data/connections.csv")
    return pd.merge(connections, classification[["root_id", "cell_type"]], left_on="pre_root_id",
                    right_on="root_id")


synapse_df = get_synapse_df()

neuron_type = 'TmY18'
num_inputs = 721
num_outputs = synapse_df[synapse_df['cell_type'] == neuron_type].shape[0]
neuron_count = synapse_df.shape[0]


# Create the integrated model
temporal_conv_net = TemporalConvNet(num_inputs, [100], num_outputs, kernel_size=2, dropout=0.2)
adult_connectome_net = AdultConnectomeNetwork(adjacency_matrix_csr, neuron_count, {"CONNECTOME_LAYER_NUMBER": 2})

integrated_model = IntegratedModel(temporal_conv_net, adult_connectome_net, synapse_df, neuron_type)

# Test the integrated model
batch_size = 2
hexal_size = 721  # Example hexal size
temporal_size = 84  # Example temporal size
input_tensor = torch.rand(batch_size, hexal_size, temporal_size)

output = integrated_model(input_tensor)
print(output.shape)

/home/eudald/Desktop/doctorat/connectome/adult_models.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  indices = torch.LongTensor([adjacency_matrix_coo.row, adjacency_matrix_coo.col])


MemoryError: Unable to allocate 5.00 EiB for an array with shape (720575940660588163,) and data type int64